# What is Computer Vision?

<img src="https://analyticsinsight.b-cdn.net/wp-content/uploads/2020/07/Computer-Vision.jpg" width="400">

Computer vision is an interdisciplinary scientific field that deals with how computers can gain high-level understanding from digital images or videos. From the perspective of engineering, it seeks to understand and automate tasks that the human visual system can do[[1](https://en.wikipedia.org/wiki/Computer_vision)]. Computer vision tasks include methods for acquiring, processing, analyzing and understanding digital images, and extraction of high-dimensional data from the real world in order to produce numerical or symbolic information, e.g. in the forms of decisions ( Reinhard Klette (2014))




# **What is image processing?**

In machine learning projects in general, you usually go through a data preprocessing or cleaning step. The goal of this step is to make your data ready for the ML model to make it easier to analyze and process computationally, as it is with images. Based on the problem you’re solving and the dataset in hand, there’s some data massaging required before you feed your images to the ML model.

Image processing could be simple tasks like image resizing.In order to feed a dataset of images to a convolutional network, they must all be the same size. Other processing tasks can take place like geometric and color transformation or converting color to grayscale and many more.

# **Why image preprocessing?**

The acquired data are usually messy and come from different sources. To feed them to the Machine Learning model (or neural network), they need to be standardized and cleaned up.


# **Data preprocessing techniques might include:**

in certain problems you’ll find it useful to lose unnecessary information from your images to reduce space or computational complexity.

For example, converting your colored images to grayscale images. This is because in many objects, color isn’t necessary to recognize and interpret an image. Grayscale can be good enough for recognizing certain objects. Because color images contain more information than black and white images, they can add unnecessary complexity and take up more space in memory (Remember how color images are represented in three channels, which means that converting it to grayscale reduces the number of pixels that need to be processed).

Practical example

In [ ]:
import os
import json

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import cv2
import albumentations as A


import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')



In [ ]:
Cassava_dir = "../input/cassava-leaf-disease-classification/"
with open(os.path.join(Cassava_dir, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    
print(json.dumps(map_classes, indent=4))

In [ ]:
input_files = os.listdir(os.path.join(Cassava_dir, "train_images"))
print(f"Number of train images: {len(input_files)}")

In [ ]:
img_shapes = {}
for image_name in os.listdir(os.path.join(Cassava_dir, "train_images"))[:300]:
    image = cv2.imread(os.path.join(Cassava_dir, "train_images", image_name))
    img_shapes[image.shape] = img_shapes.get(image.shape, 0) + 1

print(img_shapes)

In [ ]:
df = pd.read_csv(os.path.join(Cassava_dir, "train.csv"))

df["class_name"] = df["label"].astype(str).map(map_classes)

df

Read the image from the disk and convert it from the BGR color space to the RGB color space
For historical reasons, OpenCV reads an image in BGR format (so color channels of the image have the following order: Blue, Green, Red). Albumentations uses the most common and popular RGB image format. So when using OpenCV, we need to convert the image format to RGB explicitly.

In [ ]:
def show_image(image_ids, labels):
    plt.figure(figsize=(16, 12))
    
    for ind, (image_id, label) in enumerate(zip(image_ids, labels)):
        plt.subplot(3, 3, ind + 1)
        image = cv2.imread(os.path.join(Cassava_dir, "train_images", image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(f"Class: {label}", fontsize=12)
        plt.axis("off")
    
    plt.show()

In [ ]:
tmp_df = df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["class_name"].values

show_image(image_ids, labels)


# 1. Image Transform

This work tried to transform images from one color-space to another. The transformation you can see below:
* mask
* segment
* deskew
* gray
* thresh
* rnoise
* canny
* sharpen

According to [2] while the color image can be treated arbitrary vector value functions or collections of independent bands, it usually makes sense to think about them as highly correlated signals with strong connections to the image formation process, sensor design, and Human perception. Consider example brightening picture by adding a constant value to all three channels. In fact, adding the same value to each color channel not only increases the apparent intensity of each pixel, but it cal also affects the picture hue and saturation.



In [ ]:
#masking function
def create_mask_for_image(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_hsv = np.array([0,0,250])
    upper_hsv = np.array([250,255,255])
    
    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask

#image  deskew function
def  deskew_image(image):
    mask = create_mask_for_image(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255

#image  gray  function
def  gray_image(image):
    #mask = create_mask_for_image(image)
    output = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    return output/255

#image  thresh  function
def  thresh_image(image):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    output = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV) #+cv.THRESH_OTSU)
    return output


#image  rnoise  function
def  rnoise_image(image):
    mask = create_mask_for_image(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255

#image  dilate  function
def  dilate_image(image):
    mask = create_mask_for_image(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255


#image  erode  function
def  erode_image(image):
    mask = create_mask_for_image(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255



#image  opening  function
def  opening_image(image):
    mask = create_mask_for_image(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255

#image canny function
def  canny_image(image):
    mask = create_mask_for_image(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255


#image segmentation function
def segment_image(image):
    mask = create_mask_for_image(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255


#sharpen the image
def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp


# function to get an image
def read_img(image_id, labels):
    plt.figure(figsize=(16, 12))
    img = cv2.imread(os.path.join(Cassava_dir, "train_images", image_id))
    #convert image to array
    #img = image.img_to_array(img)
    return img

SHOW SAMPLE IMAGES

In [ ]:
nb_rows = 2
nb_cols = 5
fig, axs = plt.subplots(nb_rows, nb_cols, figsize=(15, 5));
plt.suptitle('SAMPLE IMAGES');
for i in range(0, nb_rows):
    for j in range(0, nb_cols):
        axs[i, j].xaxis.set_ticklabels([]);
        axs[i, j].yaxis.set_ticklabels([]);
        axs[i, j].imshow((read_img(df['image_id'][np.random.randint(1400)], (255,255)))/255.);
plt.show();

# Show Result

# mask

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#mask
image_mask = create_mask_for_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 10));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('MASK', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_mask);


In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#mask
image_mask = create_mask_for_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 10));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('MASK', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_mask);


In [ ]:
#get an image
img = read_img(df['image_id'][110],(255,255))

#mask
image_mask = create_mask_for_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 10));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('MASK', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_mask);


In [ ]:
#get an image
img = read_img(df['image_id'][1112],(255,255))

#mask
image_mask = create_mask_for_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 10));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('MASK', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_mask);


# segmented

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#segmentation
image_segmented = segment_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('SEGMENTED', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_segmented);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#segmentation
image_segmented = segment_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('SEGMENTED', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_segmented);

In [ ]:
#get an image
img = read_img(df['image_id'][5920],(255,255))

#segmentation
image_segmented = segment_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('SEGMENTED', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_segmented);

# deskew

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#deskew
image_deskew = deskew_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('DESKEW', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_deskew);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#deskew
image_deskew = deskew_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('DESKEW', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_deskew);

In [ ]:
#get an image
img = read_img(df['image_id'][21367],(255,255))

#deskew
image_deskew = deskew_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('DESKEW', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_deskew);

# gray

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#gray
image_gray = gray_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('GRAY', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_gray);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#gray
image_gray = gray_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('GRAY', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_gray);

In [ ]:
#get an image
img = read_img(df['image_id'][110],(255,255))

#gray
image_gray = gray_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('GRAY', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_gray);

# thresh

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#thresh
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU)
fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)


ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('THRESH', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(thresh);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#thresh
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU)
fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('THRESH', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(thresh);

In [ ]:
#get an image
img = read_img(df['image_id'][105],(255,255))

#thresh
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU)
fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('THRESH', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(thresh);

# rnoise

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#rnoise
image_rnoise = rnoise_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('RNOISE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_rnoise);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#rnoise
image_rnoise = rnoise_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('RNOISE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_rnoise);

In [ ]:
#get an image
img = read_img(df['image_id'][19025],(255,255))

#rnoise
image_rnoise = rnoise_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('RNOISE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_rnoise);

# canny 

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#canny
image_canny = canny_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('CANNY', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_canny);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#canny
image_canny = canny_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('CANNY', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_canny);

In [ ]:
#get an image
img = read_img(df['image_id'][6900],(255,255))

#canny
image_canny = canny_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('CANNY', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_canny);

# sharpen

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#sharpen the image
image_sharpen = sharpen_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('SHARPEN', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_sharpen);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#sharpen the image
image_sharpen = sharpen_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('SHARPEN', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_sharpen);

In [ ]:
#get an image
img = read_img(df['image_id'][2090],(255,255))

#sharpen the image
image_sharpen = sharpen_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('SHARPEN', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_sharpen);

# Comparison of color space transformation

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#mask
image_mask = create_mask_for_image(img)

#segmentation
image_segmented = segment_image(img)


#deskew
image_deskew = deskew_image(img)

#gray
image_gray = gray_image(img)



fig, ax = plt.subplots(1, 5, figsize=(15, 6));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(2)

ax[0].set_title('ORIGINAL', fontsize=9)
ax[1].set_title('MASK', fontsize=9)
ax[2].set_title('SEGMENTED', fontsize=9)
ax[3].set_title('DESKEW', fontsize=9)
ax[4].set_title('GRAY', fontsize=9)


ax[0].imshow(img/255);
ax[1].imshow(image_mask);
ax[2].imshow(image_segmented);
ax[3].imshow(image_deskew);
ax[4].imshow(image_gray );



In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(15, 6));
plt.tight_layout(2)

#thresh
image_thresh = thresh_image(img)

#rnoise
image_rnoise = rnoise_image(img)

#canny
image_canny = canny_image(img)

#sharpen the image
image_sharpen = sharpen_image(img)


ax[0].set_title('ORIGINAL', fontsize=9)
ax[1].set_title('THREST', fontsize=9)
ax[2].set_title('RNOISE', fontsize=9)
ax[3].set_title('CANNY', fontsize=9)
ax[4].set_title('SHARPEN', fontsize=9)


ax[0].imshow(img/255);
ax[1].imshow(thresh);
ax[2].imshow(image_rnoise);
ax[3].imshow(image_canny);
ax[4].imshow(image_sharpen);





# 2. Morphological Transformations

# erosion

The basic idea of erosion is just like soil erosion only, it erodes away the boundaries of foreground object (Always try to keep foreground in white)

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#erode
image_erode = erode_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('ERODE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_erode);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#erode
image_erode = erode_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('ERODE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_erode);

In [ ]:
#get an image
img = read_img(df['image_id'][110],(255,255))

#erode
image_erode = erode_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('ERODE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_erode);

# dilate

It is just opposite of erosion. Here, a pixel element is ‘1’ if atleast one pixel under the kernel is ‘1’. So it increases the white region in the image or size of foreground object increases. Normally, in cases like noise removal, erosion is followed by dilation.

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#dilate
image_dilate = dilate_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('DILATE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_dilate);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#dilate
image_dilate = dilate_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('DILATE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_dilate);

In [ ]:
#get an image
img = read_img(df['image_id'][110],(255,255))

#dilate
image_dilate = dilate_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('DILATE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_dilate);

# Opening

Opening is just another name of erosion followed by dilation. It is useful in removing noise

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#opening
image_opening = opening_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('OPENING', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_opening);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#opening
image_opening = opening_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('OPENING', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_opening);

In [ ]:
#get an image
img = read_img(df['image_id'][110],(255,255))

#opening
image_opening = opening_image(img)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('OPENING', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(image_opening);

# Comparison of Morphological Transformations

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#dilate
image_dilate = dilate_image(img)

#erode
image_erode = erode_image(img)

#opening
image_opening = opening_image(img)

fig, ax = plt.subplots(1, 4, figsize=(15, 6));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=9)
ax[1].set_title('DILATE', fontsize=9)
ax[2].set_title('ERODE', fontsize=9)
ax[3].set_title('OPENING', fontsize=9)


ax[0].imshow(img/255);
ax[1].imshow(image_dilate);
ax[2].imshow(image_erode);
ax[3].imshow(image_opening);


# 3. Image Blurring (Image Smoothing)

Image blurring is achieved by convolving the image with a low-pass filter kernel. It is useful for removing noise. It actually removes high frequency content (e.g: noise, edges) from the image resulting in edges being blurred when this is filter is applied. (Well, there are blurring techniques which do not blur edges). OpenCV provides mainly four types of blurring techniques.

# Averaging

This is done by convolving the image with a normalized box filter. It simply takes the average of all the pixels under kernel area and replaces the central element with this average. This is done by the function cv2.blur() or cv2.boxFilter(). 

In [ ]:

#get an image
img = read_img(df['image_id'][13],(255,255))

#Blur
blur = cv2.blur(img,(5,5))

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('BLUR IMAGE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(blur);

In [ ]:

#get an image
img = read_img(df['image_id'][15],(255,255))

#Blur
blur = cv2.blur(img,(5,5))

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('BLUR IMAGE', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(blur);

# Gaussian Filtering
It is done with the function, cv2.GaussianBlur(). 

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#GaussianBlur
Gblur = cv2.GaussianBlur(img,(5,5),0)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('GAUSSIAN BLUR', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(Gblur);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#GaussianBlur
Gblur = cv2.GaussianBlur(img,(5,5),0)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('GAUSSIAN BLUR', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(Gblur);

# Median Filtering

Here, the function cv2.medianBlur() computes the median of all the pixels under the kernel window and the central pixel is replaced with this median value. This is highly effective in removing salt-and-pepper noise.

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

#medianBlur
blur_image_median = cv2.medianBlur(img,5)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('MEDIAN BLUR', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(blur_image_median);

In [ ]:
#get an image
img = read_img(df['image_id'][15],(255,255))

#medianBlur
blur_image_median = cv2.medianBlur(img,5)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('MEDIAN BLUR', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(blur_image_median);

# Bilateral Filtering

According to [3] cv2.bilateralFilter() is highly effective at noise removal while preserving edges

In [ ]:

#get an image
img = read_img(df['image_id'][13],(255,255))

#BILATERAL FILTER
bilblur = cv2.bilateralFilter(img,9,75,75)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('BILATERAL FILTER', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(bilblur);


In [ ]:

#get an image
img = read_img(df['image_id'][15],(255,255))

#BILATERAL FILTER
bilblur = cv2.bilateralFilter(img,9,75,75)

fig, ax = plt.subplots(1, 2, figsize=(15, 5));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=12)
ax[1].set_title('BILATERAL FILTER', fontsize=12)

ax[0].imshow(img/255);
ax[1].imshow(bilblur);


# Comparison of Image Blurring

In [ ]:
#get an image
img = read_img(df['image_id'][13],(255,255))

fig, ax = plt.subplots(1, 5, figsize=(15, 6));
plt.suptitle('RESULT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIGINAL', fontsize=9)
ax[1].set_title('BLUR IMAGE', fontsize=9)
ax[2].set_title('GAUSSIAN BLUR', fontsize=9)
ax[3].set_title('MEDIAN BLUR', fontsize=9)
ax[4].set_title('BILATERAL FILTER', fontsize=9)


ax[0].imshow(img/255);
ax[1].imshow(blur);
ax[2].imshow(Gblur);
ax[3].imshow(blur_image_median);
ax[4].imshow(bilblur);


## Rotating image based on a specific angle


In the code below, the image is rotated in increments of 60-degrees using rotate() of imutils


In [ ]:
!pip install imutils


In [ ]:
import imutils
import numpy as np
import cv2

image = image= cv2.imread(r'../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')

# loop over the rotation angles
for angle in np.arange(0, 360, 60):
    cv2.namedWindow("Rotated", cv2.WINDOW_NORMAL)
    rotated = imutils.rotate(image, angle)
    cv2.imshow("Rotated", rotated)
    cv2.waitKey(0)

## Rotating image using PIL

Here the image is rotated by 110 degrees using PIL

In [ ]:
import cv2

# Rotate image using PIL
#image= cv2.imread(r'../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
pil_image= cv2.imread(r'../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
rotate_img_pil=pil_image.rotate(110)
rotate_img_pil.show()

# References

[1] https://en.wikipedia.org/wiki/Computer_vision

[2] http://szeliski.org/Book/

[3] https://opencv-python-tutroals.readthedocs.io/


